# Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import requests
from sklearn import datasets

# Load CSV File

In [2]:
df_csv = pd.read_csv('/Users/jshn/Desktop/UPTRAIL INTERNSHIP/Project Files/customer_signups.csv')
print("First 5 rows:")
print(df_csv.head())

First 5 rows:
  customer_id             name                 email signup_date     source  \
0   CUST00000    Joshua Bryant                   NaN         NaN  Instagram   
1   CUST00001   Nicole Stewart   nicole1@example.com    02-01-24   LinkedIn   
2   CUST00002     Rachel Allen   rachel2@example.com    03-01-24     Google   
3   CUST00003  Zachary Sanchez  zachary3@mailhub.org    04-01-24    YouTube   
4   CUST00004              NaN  matthew4@mailhub.org    05-01-24   LinkedIn   

  region plan_selected marketing_opt_in age      gender  
0    NaN         basic               No  34      Female  
1   West         basic              Yes  29        Male  
2  North       PREMIUM              Yes  34  Non-Binary  
3    NaN           Pro               No  40        Male  
4   West       Premium               No  25       Other  


# 1. Load & Clean the Data

In [3]:
#Identify Missing Values 
df_csv.isnull()
print(df_csv.isnull().sum())

customer_id          2
name                 9
email               34
signup_date          2
source               9
region              30
plan_selected        8
marketing_opt_in    10
age                 12
gender               8
dtype: int64


In [4]:
#Data Types
print(df_csv.dtypes)

customer_id         object
name                object
email               object
signup_date         object
source              object
region              object
plan_selected       object
marketing_opt_in    object
age                 object
gender              object
dtype: object


In [5]:
#Column Structure
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer_id       298 non-null    object
 1   name              291 non-null    object
 2   email             266 non-null    object
 3   signup_date       298 non-null    object
 4   source            291 non-null    object
 5   region            270 non-null    object
 6   plan_selected     292 non-null    object
 7   marketing_opt_in  290 non-null    object
 8   age               288 non-null    object
 9   gender            292 non-null    object
dtypes: object(10)
memory usage: 23.6+ KB


In [6]:
#Convert Signup_date To Datetime
df_csv.rename(columns={'signup_date': 'datetime'}, inplace=True)
print(df_csv)

    customer_id                name                  email  datetime  \
0     CUST00000       Joshua Bryant                    NaN       NaN   
1     CUST00001      Nicole Stewart    nicole1@example.com  02-01-24   
2     CUST00002        Rachel Allen    rachel2@example.com  03-01-24   
3     CUST00003     Zachary Sanchez   zachary3@mailhub.org  04-01-24   
4     CUST00004                 NaN   matthew4@mailhub.org  05-01-24   
..          ...                 ...                    ...       ...   
295   CUST00295          Gary Smith     gary95@example.com  22-10-24   
296   CUST00296     Anthony Roberts  anthony96@mailhub.org  23-10-24   
297   CUST00297  Timothy Mclaughlin                    NaN  24-10-24   
298   CUST00298     Justin Mcintyre   justin98@mailhub.org  25-10-24   
299   CUST00299   Mr. Bruce Bridges      mr.99@example.com  26-10-24   

        source   region plan_selected marketing_opt_in age      gender  
0    Instagram      NaN         basic               No  34    

In [19]:
#Standardise Inconsistent Text Values
# Define mappings
plan_mapping = {
    'basic': 'Basic',
    'premium': 'Premium',
    'pro': 'Pro'
}

gender_mapping = {
    'male': 'Male',
    'female': 'Female',
    'non-binary': 'Non-Binary',
    'other': 'Other'
}

# Clean plan_selected
df_csv['plan_selected'] = (
    df_csv['plan_selected']
    .astype(str)
    .str.strip()
    .str.lower()
    .map(plan_mapping)
)

# Clean gender
df_csv['gender'] = (
    df_csv['gender']
    .astype(str)
    .str.strip()
    .str.lower()
    .map(gender_mapping)
)

print("Unique plan values:", df_csv['plan_selected'].unique())
print("Unique gender values:", df_csv['gender'].unique())

Unique plan values: ['Basic' 'Premium' 'Pro' nan]
Unique gender values: ['Female' 'Male' 'Non-Binary' 'Other' nan]


In [15]:
#Removing Duplicate Rows Based on Customer Id
df_csv = df_csv.drop_duplicates(subset='customer_id', keep='first')
print("\Cleaned DataFrame:\n", df_csv)

\Cleaned DataFrame:
     customer_id                name                  email  datetime  \
0     CUST00000       Joshua Bryant                    NaN       NaN   
1     CUST00001      Nicole Stewart    nicole1@example.com  02-01-24   
2     CUST00002        Rachel Allen    rachel2@example.com  03-01-24   
3     CUST00003     Zachary Sanchez   zachary3@mailhub.org  04-01-24   
4     CUST00004                 NaN   matthew4@mailhub.org  05-01-24   
..          ...                 ...                    ...       ...   
295   CUST00295          Gary Smith     gary95@example.com  22-10-24   
296   CUST00296     Anthony Roberts  anthony96@mailhub.org  23-10-24   
297   CUST00297  Timothy Mclaughlin                    NaN  24-10-24   
298   CUST00298     Justin Mcintyre   justin98@mailhub.org  25-10-24   
299   CUST00299   Mr. Bruce Bridges      mr.99@example.com  26-10-24   

        source   region plan_selected marketing_opt_in age      gender  
0    Instagram      NaN         Basic    

In [16]:
#Handling Missing Values(region, email, age)

#Missing region
df_csv['region'] = df_csv['region'].fillna('Unknown')

#Missing email
df_csv['email'] = df_csv['email'].fillna('Not_Provided')

#Missing age 
df_csv['age'] = df_csv['age'].fillna(0)

# 2. Data Quality Summary

In [24]:
# Count Missing Values Per Column 
missing_values_count = df_csv.isnull().sum()
missing_values = missing_values_count[missing_values_count > 0]  

print(missing_values)

customer_id          1
name                 9
datetime             2
source               9
plan_selected       14
marketing_opt_in    10
gender              14
dtype: int64


In [25]:
# % Of Missing Values
missing_percentage = df_csv.isnull().mean() * 100

print(missing_percentage)

customer_id         0.334448
name                3.010033
email               0.000000
datetime            0.668896
source              3.010033
region              0.000000
plan_selected       4.682274
marketing_opt_in    3.344482
age                 0.000000
gender              4.682274
dtype: float64


In [27]:
#Number Of Duplicates Removed 
initial_rows = len(df_csv)

#Drop Duplicates
df_csv = df_csv.drop_duplicates()

final_rows = len(df_csv)

#Calculate Numbers Of Duplicates Removed 
duplicates_removed = initial_rows - final_rows

print(f"Number of duplicate rows removed: {duplicates_removed}")

Number of duplicate rows removed: 0


In [30]:
#Inconsistent Category Values Corrected 
print("Unique values before cleaning:")
print(df_csv['gender'].unique())

Unique values before cleaning:
['Female' 'Male' 'Non-Binary' 'Other' nan]


# 3. Summary Outputs (Using Pandas Aggregations)

In [42]:
#Sign-ups per week

# Convert signup_date to datetime
df_csv['datetime'] = pd.to_datetime(df_csv['datetime'], errors='coerce')

# Group by week and count signups
signups_per_week = df_csv.groupby(pd.Grouper(key='datetime', freq='W')).size()

# Print results
print(signups_per_week)


datetime
2024-01-07    6
2024-01-14    5
2024-01-21    7
2024-01-28    7
2024-02-04    8
2024-02-11    6
2024-02-18    6
2024-02-25    7
2024-03-03    7
2024-03-10    7
2024-03-17    5
2024-03-24    6
2024-03-31    6
2024-04-07    7
2024-04-14    5
2024-04-21    7
2024-04-28    7
2024-05-05    6
2024-05-12    4
2024-05-19    7
2024-05-26    7
2024-06-02    7
2024-06-09    7
2024-06-16    5
2024-06-23    7
2024-06-30    7
2024-07-07    7
2024-07-14    5
2024-07-21    6
2024-07-28    7
2024-08-04    7
2024-08-11    5
2024-08-18    6
2024-08-25    7
2024-09-01    7
2024-09-08    7
2024-09-15    5
2024-09-22    7
2024-09-29    7
2024-10-06    7
2024-10-13    5
2024-10-20    6
2024-10-27    6
2024-11-03    3
2024-11-10    7
2024-11-17    0
2024-11-24    0
2024-12-01    1
2024-12-08    7
2024-12-15    2
dtype: int64


/var/folders/8c/f33hbs394s3b3wd1k7v6ywc40000gn/T/ipykernel_61576/548172622.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_csv['datetime'] = pd.to_datetime(df_csv['datetime'], errors='coerce')


In [45]:
#Sign-ups by source, region, and plan_selected
signups_summary = df_csv[['source', 'region', 'plan_selected']].value_counts().reset_index(name='count')

print(signups_summary)

        source   region plan_selected  count
0      YouTube    South         Basic      9
1      YouTube     East         Basic      8
2       Google    North       Premium      7
3    Instagram    North       Premium      6
4     LinkedIn    South         Basic      6
..         ...      ...           ...    ...
98    LinkedIn  Central           Pro      1
99    LinkedIn     East       Premium      1
100   LinkedIn  Unknown       Premium      1
101   Referral  Central       Premium      1
102    YouTube     West           Pro      1

[103 rows x 4 columns]


In [47]:
#Marketing opt-in count by gender
marketing_optin_counts = df_csv.groupby(['gender', 'marketing_opt_in']).size().reset_index(name='count')

print(marketing_optin_counts)

       gender marketing_opt_in  count
0      Female               No     47
1      Female              Yes     44
2        Male              Nil      1
3        Male               No     50
4        Male              Yes     38
5  Non-Binary               No     20
6  Non-Binary              Yes     19
7       Other               No     32
8       Other              Yes     24


In [49]:
#Age summary:min, max, mean, median, null count
df_csv['age'] = pd.to_numeric(df_csv['age'], errors='coerce')  # 'coerce' will convert non-numeric values to NaN

age_summary = {
    'min': df_csv['age'].min(),
    'max': df_csv['age'].max(),
    'mean': df_csv['age'].mean(),
    'median': df_csv['age'].median(),
    'null_count': df_csv['age'].isnull().sum()
}

print(age_summary)

{'min': 0.0, 'max': 206.0, 'mean': np.float64(34.68835616438356), 'median': 34.0, 'null_count': np.int64(7)}
